## 10.1 GroupBy 機制

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.DataFrame({'key1': ['a', 'a', 'b', 'b', 'a'],
                   'key2': ['one', 'two', 'one', 'two', 'one'],
                   'data1': np.random.randn(5),
                   'data2': np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,0.567809,-0.132201
1,a,two,0.091732,-1.047888
2,b,one,-0.109994,0.408990
3,b,two,1.288401,1.114073
4,a,one,0.234972,0.168156


In [5]:
# 以 key1 的標籤做分組，計算 data1 的分組平均值
grouped = df['data1'].groupby(df['key1'])
grouped.mean()

key1
a    0.663384
b    0.735482
Name: data1, dtype: float64

In [6]:
# key為多個陣列組成的 list
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

key1  key2
a     one     0.544489
      two     0.901173
b     one     0.742818
      two     0.728147
Name: data1, dtype: float64

In [7]:
means.unstack()

key2,one,two
key1,,
a,0.544489,0.901173
b,0.742818,0.728147


In [9]:
# key為 Series
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])

df['data1'].groupby([states, years]).mean()

California  2005    0.901173
            2006    0.742818
Ohio        2005    0.690598
            2006    0.435930
Name: data1, dtype: float64

In [11]:
# 輸入欄位名稱當作分組 key
df.groupby('key1').mean()

,data1,data2
key1,,
a,0.663384,0.171832
b,0.735482,-0.479382


In [12]:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one   0.544489  0.012059
     two   0.901173  0.491377
b    one   0.742818 -0.225516
     two   0.728147 -0.733247

In [13]:
# 利用 groupby 計算各分組的大小
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

### 疊代分組

In [4]:
for name, group in df.groupby('key1'):
    print(name)
    print(group)
    print('---')

a
  key1 key2     data1     data2
0    a  one -0.031661 -0.004566
1    a  two  0.143277  2.125950
4    a  one  1.635402  1.599679
---
b
  key1 key2     data1     data2
2    b  one -0.669080 -0.495781
3    b  two  2.822098  1.347192
---


In [5]:
for (k1 ,k2), group in df.groupby(['key1', 'key2']):
    print((k1, k2))
    print(group)
    print('---')

('a', 'one')
  key1 key2     data1     data2
0    a  one -0.031661 -0.004566
4    a  one  1.635402  1.599679
---
('a', 'two')
  key1 key2     data1    data2
1    a  two  0.143277  2.12595
---
('b', 'one')
  key1 key2    data1     data2
2    b  one -0.66908 -0.495781
---
('b', 'two')
  key1 key2     data1     data2
3    b  two  2.822098  1.347192
---


In [12]:
# list(df.groupby('key1'))
piece = dict(list(df.groupby('key1')))
piece['b']

,key1,key2,data1,data2
2,b,one,-0.669080,-0.495781
3,b,two,2.822098,1.347192


In [13]:
# 指定做欄分組
df.dtypes

key1      object
key2      object
data1    float64
data2    float64
dtype: object

In [15]:
grouped = df.groupby(df.dtypes, axis=1)
grouped

In [16]:
for dtype, group in grouped:
    print(dtype)
    print(group)
    print('---')

float64
      data1     data2
0 -0.031661 -0.004566
1  0.143277  2.125950
2 -0.669080 -0.495781
3  2.822098  1.347192
4  1.635402  1.599679
---
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one
---


### 選取一個或多個欄

In [17]:
df

,key1,key2,data1,data2
0,a,one,-0.031661,-0.004566
1,a,two,0.143277,2.125950
2,b,one,-0.669080,-0.495781
3,b,two,2.822098,1.347192
4,a,one,1.635402,1.599679


In [19]:
# 計算 data2 欄的分組平均，結果以 DataFrame回傳 <-- 參數傳遞 list/array
df.groupby(['key1', 'key2'])[['data2']].mean()

data2
key1 key2          
a    one   0.797557
     two   2.125950
b    one  -0.495781
     two   1.347192

In [21]:
# 計算 data2 欄的分組平均，結果以 Series 回傳 <-- 參數傳遞 常數欄位名稱

df.groupby(['key1', 'key2'])['data2'].mean()

key1  key2
a     one     0.797557
      two     2.125950
b     one    -0.495781
      two     1.347192
Name: data2, dtype: float64

### 用Dict和Series進行分組

In [22]:
people = pd.DataFrame(np.random.randn(5,5),
                      columns=['a', 'b', 'c', 'd', 'e'],
                      index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people

,a,b,c,d,e
Joe,0.745578,-0.413657,-0.075199,-0.414638,-1.416007
Steve,1.571365,-0.028639,-3.004358,2.240465,-0.740359
Wes,1.243278,0.224609,-1.677511,-0.315819,-0.230073
Jim,-1.168506,-1.231021,-0.420512,2.132458,-0.200163
Travis,0.853071,-0.701867,0.337946,0.629953,1.018118


In [26]:
people.iloc[2:3, [1,2]] = np.nan # 加入幾個 NA值

In [27]:
people

,a,b,c,d,e
Joe,0.745578,-0.413657,-0.075199,-0.414638,-1.416007
Steve,1.571365,-0.028639,-3.004358,2.240465,-0.740359
Wes,1.243278,NaN,NaN,-0.315819,-0.230073
Jim,-1.168506,-1.231021,-0.420512,2.132458,-0.200163
Travis,0.853071,-0.701867,0.337946,0.629953,1.018118


In [28]:
# 分組對應表
mapping = {'a': 'red', 'b': 'red', 'c': 'blue', 'd': 'blue', 'e': 'red', 'f': 'orange'}

In [29]:
by_column = people.groupby(mapping, axis=1)
by_column

In [30]:
by_column.sum()

,blue,red
Joe,-0.489837,-1.084086
Steve,-0.763892,0.802367
Wes,-0.315819,1.013205
Jim,1.711946,-2.599690
Travis,0.967898,1.169323


In [31]:
# 用Series分組
map_series = pd.Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [32]:
people.groupby(map_series, axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


### 用函式分組

In [33]:
people

,a,b,c,d,e
Joe,0.745578,-0.413657,-0.075199,-0.414638,-1.416007
Steve,1.571365,-0.028639,-3.004358,2.240465,-0.740359
Wes,1.243278,NaN,NaN,-0.315819,-0.230073
Jim,-1.168506,-1.231021,-0.420512,2.132458,-0.200163
Travis,0.853071,-0.701867,0.337946,0.629953,1.018118


In [34]:
# 以名字長度做分組
people.groupby(len).sum()

,a,b,c,d,e
3,0.820350,-1.644678,-0.495711,1.402001,-1.846243
5,1.571365,-0.028639,-3.004358,2.240465,-0.740359
6,0.853071,-0.701867,0.337946,0.629953,1.018118


In [35]:
# 函式混搭 array/dict.Series
key_list = ['one', 'one', 'one', 'two', 'two']
people.groupby([len, key_list]).min()    # axis: 'Axis' = 0

a         b         c         d         e
3 one  0.745578 -0.413657 -0.075199 -0.414638 -1.416007
  two -1.168506 -1.231021 -0.420512  2.132458 -0.200163
5 one  1.571365 -0.028639 -3.004358  2.240465 -0.740359
6 two  0.853071 -0.701867  0.337946  0.629953  1.018118

### 用索引層級分組

In [36]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
                                     [1, 3, 5, 1, 3]],
                                    names=['city', 'tenor'])
columns

MultiIndex([('US', 1),
            ('US', 3),
            ('US', 5),
            ('JP', 1),
            ('JP', 3)],
           names=['city', 'tenor'])

In [37]:
hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)
hier_df

city         US                            JP          
tenor         1         3         5         1         3
0      0.309312  1.548185 -0.176203 -2.057458 -1.774331
1     -0.722857 -0.756337  0.056873 -1.470106  0.085368
2     -0.115956 -0.963551  0.564273  1.462182 -0.206158
3      0.061872 -1.010317 -0.583031 -0.939967 -0.262382

In [38]:
hier_df.groupby(level='city', axis=1).count()

city,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


---

## 10.2 資料聚合

#### 任何對資料矩陣做轉換，得到一些常數的動作，就稱為聚合。

In [3]:
df

,key1,key2,data1,data2
0,a,one,0.567809,-0.132201
1,a,two,0.091732,-1.047888
2,b,one,-0.109994,0.408990
3,b,two,1.288401,1.114073
4,a,one,0.234972,0.168156


In [6]:
grouped = df.groupby('key1')
grouped

In [8]:
grouped['data1'].quantile(0.9)   # 計算欄位的百分位數

key1
a    0.501242
b    1.148561
Name: data1, dtype: float64

In [31]:
# 使用自己的聚合函式
def peak_to_peak(arr):
    return arr.max() - arr.min()

grouped.agg(peak_to_peak)  # 當作參數傳遞給 agg/aggregate

C:\Program Files (x86)\Python38-64\lib\site-packages\pandas\core\groupby\generic.py:303: FutureWarning: Dropping invalid columns in SeriesGroupBy.agg is deprecated. In a future version, a TypeError will be raised. Before calling .agg, select only columns which should be valid for the aggregating function.
  results[key] = self.aggregate(func)


,data1,data2
key1,,
a,0.476077,1.216044
b,1.398394,0.705083


In [38]:
# ############################
# 怎麼改???
# ############################

df2 = pd.DataFrame({'key1': [1, 1, 2, 2, 1],
                    'key2': ['one', 'two', 'one', 'two', 'one'],
                    'data1': np.random.randn(5),
                    'data2': np.random.randn(5)})

def peak_to_peak(arr):
    return arr.max() - arr.min()

df2.groupby('key1').key1.agg(peak_to_peak)

key1
1    0
2    0
Name: key1, dtype: int64

In [15]:
from pandas import *
d = {"series": Series(['1','2','1','1','4','4','5'])}
dfex = DataFrame(d)
def get_count(values):
    return len(values)
dfex.groupby("series").series.agg(get_count)


series
1    3
2    1
4    2
5    1
Name: series, dtype: int64


In [40]:
grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0  0.298171  0.244250  0.091732  0.163352  0.234972  0.401391   
b      2.0  0.589203  0.988814 -0.109994  0.239605  0.589203  0.938802   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     0.567809   3.0 -0.337311  0.633438 -1.047888 -0.590045 -0.132201   
b     1.288401   2.0  0.761532  0.498569  0.408990  0.585261  0.761532   

                          
           75%       max  
key1                      
a     0.017978  0.168156  
b     0.937803  1.114073

### 欄方向的多功能應用

In [2]:
tips = pd.read_csv('examples/tips.csv')

In [46]:
# tips.keys()
tips.head()

,total_bill,tip,smoker,day,time,size
0,16.99,1.01,No,Sun,Dinner,2
1,10.34,1.66,No,Sun,Dinner,3
2,21.01,3.50,No,Sun,Dinner,3
3,23.68,3.31,No,Sun,Dinner,2
4,24.59,3.61,No,Sun,Dinner,4


In [3]:
tips['tip_pct'] = tips['tip'] / tips['total_bill']
tips.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


In [51]:
# 先進行分組
grouped = tips.groupby(['day', 'smoker'])

In [52]:
grouped_pct = grouped['tip_pct']

In [53]:
# 傳入一個函式名稱字串
grouped_pct.agg('mean')

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [54]:
# 傳入一串函式名稱
grouped_pct.agg(['mean', 'std', peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

In [55]:
# 傳遞(name, function)tuple
grouped_pct.agg([('foo', 'mean'), ('bar', np.std)])

foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

In [56]:
# 指定函式 list，對不同的欄套用不同函式
function = ['count', 'mean', 'max']
result = grouped['tip_pct', 'total_bill'].agg(function)
result

C:\Users\a8803\AppData\Local\Temp/ipykernel_9692/3518771023.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  result = grouped['tip_pct', 'total_bill'].agg(function)


tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [61]:
# 同理
result['tip_pct']

count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

In [62]:
# 傳入 tuple 組成的 list來自定名稱
ftuples = [('Durchschnitt', 'mean'), ('Abweichung', np.var)]
grouped['tip_pct', 'total_bill'].agg(ftuples)

C:\Users\a8803\AppData\Local\Temp/ipykernel_9692/1625430256.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  grouped['tip_pct', 'total_bill'].agg(ftuples)


tip_pct              total_bill            
            Durchschnitt Abweichung Durchschnitt  Abweichung
day  smoker                                                 
Fri  No         0.151650   0.000791    18.420000   25.596333
     Yes        0.174783   0.002631    16.813333   82.562438
Sat  No         0.158048   0.001581    19.661778   79.908965
     Yes        0.147906   0.003767    21.276667  101.387535
Sun  No         0.160113   0.001793    20.506667   66.099980
     Yes        0.187250   0.023757    24.120000  109.046044
Thur No         0.160298   0.001503    17.113111   59.625081
     Yes        0.163863   0.001551    19.190588   69.808518

In [65]:
# 將不同函式，套用到一或多個欄位上
grouped.agg({'tip': np.max, 'size': 'sum'})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [66]:
grouped.agg({'tip_pct': ['min', 'max', 'mean', 'std'],
             'size': 'sum'})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

### 回傳聚合資料時不要列索引

In [67]:
tips.groupby(['day', 'smoker'], as_index=False).mean()

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


In [73]:
tips.groupby(['day', 'smoker']).mean().reset_index

<bound method DataFrame.reset_index of              total_bill       tip      size   tip_pct
day  smoker                                          
Fri  No       18.420000  2.812500  2.250000  0.151650
     Yes      16.813333  2.714000  2.066667  0.174783
Sat  No       19.661778  3.102889  2.555556  0.158048
     Yes      21.276667  2.875476  2.476190  0.147906
Sun  No       20.506667  3.167895  2.929825  0.160113
     Yes      24.120000  3.516842  2.578947  0.187250
Thur No       17.113111  2.673778  2.488889  0.160298
     Yes      19.190588  3.030000  2.352941  0.163863>

---

## 10.3 Allpy: 分裂-套用-合併

In [8]:
# 功能: 選取一某條件分組後的tip_pct值前五名
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column)[-n:]

top(tips, n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


In [7]:
tips.sort_values(by='tip_pct')[-5:]

,total_bill,tip,smoker,day,time,size,tip_pct
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


In [9]:
# 指定smoker分組，並傳入apply中
tips.groupby('smoker').apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

In [11]:
# 如果有其他的參數或關鍵字要傳
tips.groupby(['smoker', 'day']).apply(top, n=1, column='total_bill')

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

In [14]:
# describe
result = tips.groupby('smoker')['tip_pct'].describe()
result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [15]:
result.unstack('smoker')

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

In [16]:
### 消除分組key索引

In [17]:
tips.groupby('smoker', group_keys=False).apply(top)

,total_bill,tip,smoker,day,time,size,tip_pct
88,24.71,5.85,No,Thur,Lunch,2,0.236746
185,20.69,5.00,No,Sun,Dinner,5,0.241663
51,10.29,2.60,No,Sun,Dinner,2,0.252672
149,7.51,2.00,No,Thur,Lunch,2,0.266312
232,11.61,3.39,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


### 分位數與籃(bucket)分析

In [18]:
frame = pd.DataFrame({'data1': np.random.randn(1000),
                      'data2': np.random.randn(1000)})

In [20]:
quartiles = pd.cut(frame.data1, 4)
quartiles[:10]

0      (-1.529, 0.12]
1    (-3.185, -1.529]
2        (0.12, 1.77]
3        (0.12, 1.77]
4        (0.12, 1.77]
5        (0.12, 1.77]
6        (0.12, 1.77]
7      (-1.529, 0.12]
8        (0.12, 1.77]
9        (0.12, 1.77]
Name: data1, dtype: category
Categories (4, interval[float64, right]): [(-3.185, -1.529] < (-1.529, 0.12] < (0.12, 1.77] < (1.77, 3.419]]

In [21]:
# 為 data2欄計算每個籃的統計值, 籃為相等長度
def get_stats(group):
    return {'min': group.min(),
            'max': group.max(),
            'count': group.count(),
            'mean': group.mean()}

In [24]:
grouped = frame.data2.groupby(quartiles)

grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
"(-3.185, -1.529]",-2.334477,2.134212,56.0,-0.024869
"(-1.529, 0.12]",-2.992966,2.875780,476.0,0.008793
"(0.12, 1.77]",-3.455231,3.181211,423.0,0.022254
"(1.77, 3.419]",-2.564441,1.997402,45.0,0.114981


In [29]:
# 籃為相等數量
grouping = pd.qcut(frame.data1, 10, labels=False)

grouping # 回傳百分位數

0      5
1      0
2      6
3      7
4      8
      ..
995    7
996    3
997    6
998    3
999    3
Name: data1, Length: 1000, dtype: int64

In [36]:
grouped = frame.data2.groupby(grouping)  # <pandas.core.groupby.generic.SeriesGroupBy object at 0x000001CEA515C550>
grouped.describe()

,count,mean,std,min,25%,50%,75%,max
data1,,,,,,,,
0,100.0,-0.094755,1.068004,-2.992966,-0.677445,-0.197200,0.750059,2.134212
1,100.0,0.039425,1.036332,-1.967146,-0.623957,0.174797,0.972957,1.963739
2,100.0,-0.034437,1.039028,-2.158064,-0.869029,0.074565,0.655237,1.972458
3,100.0,0.034377,0.939427,-2.167948,-0.547685,0.057217,0.505848,2.694716
4,100.0,0.103468,1.026344,-2.240738,-0.560863,0.113188,0.694063,2.875780
5,100.0,0.002162,1.027668,-3.106101,-0.543869,0.116970,0.644580,2.907118
6,100.0,-0.052429,1.073471,-2.648918,-0.840727,-0.054244,0.507626,2.608753
7,100.0,0.087555,0.898698,-2.394700,-0.478963,-0.050033,0.612786,2.756173
8,100.0,-0.011377,1.084680,-3.455231,-0.631314,0.096183,0.765468,3.181211


In [38]:
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
0,-2.992966,2.134212,100.0,-0.094755
1,-1.967146,1.963739,100.0,0.039425
2,-2.158064,1.972458,100.0,-0.034437
3,-2.167948,2.694716,100.0,0.034377
4,-2.240738,2.875780,100.0,0.103468
5,-3.106101,2.907118,100.0,0.002162
6,-2.648918,2.608753,100.0,-0.052429
7,-2.394700,2.756173,100.0,0.087555
8,-3.455231,3.181211,100.0,-0.011377


### 範例：依分組指定填充遺失值

In [39]:
s = pd.Series(np.random.randn(6))
s

0    1.053478
1   -1.519571
2   -0.691350
3    0.587751
4   -1.030620
5   -0.336813
dtype: float64

In [40]:
s[::2] = np.nan
s

0         NaN
1   -1.519571
2         NaN
3    0.587751
4         NaN
5   -0.336813
dtype: float64

In [41]:
s.fillna(s.mean())

0   -0.422878
1   -1.519571
2   -0.422878
3    0.587751
4   -0.422878
5   -0.336813
dtype: float64

In [43]:
# 假設:需要依照不同分組，填充不同的值
states = ['Ohio', 'New York', 'Vermont', 'Florida', 'Oregon', 'Nevada', 'California', 'Idaho']
group_key = ['East'] * 4 + ['West'] * 4   # <------ 將 East中的元素複製 4份,並產生包含此 4份元素的 list
data = pd.Series(np.random.randn(8), index=states)
data

Ohio          1.954930
New York      0.291042
Vermont       1.077184
Florida       1.657658
Oregon       -1.331981
Nevada        0.053831
California    1.006778
Idaho         0.048795
dtype: float64

In [44]:
data[['Vermont', 'Nevada', 'Idaho']] = np.nan
data

Ohio          1.954930
New York      0.291042
Vermont            NaN
Florida       1.657658
Oregon       -1.331981
Nevada             NaN
California    1.006778
Idaho              NaN
dtype: float64

In [45]:
data.groupby(group_key).mean()

East    1.301210
West   -0.162601
dtype: float64

In [47]:
# 使用 分組平均 做遺失值填充
fill_mean = lambda g: g.fillna(g.mean())
fill_mean

<function __main__.<lambda>(g)>

In [48]:
data.groupby(group_key).apply(fill_mean)

Ohio          1.954930
New York      0.291042
Vermont       1.301210
Florida       1.657658
Oregon       -1.331981
Nevada       -0.162601
California    1.006778
Idaho        -0.162601
dtype: float64

In [50]:
# 為不同分組寫一些填充值
fill_values = {'East': 0.5, 'West': -1}

fill_func = lambda g: g.fillna(fill_values[g.name])   # name: 分組時內部產生的 attribute

data.groupby(group_key).apply(fill_func)

Ohio          1.954930
New York      0.291042
Vermont       0.500000
Florida       1.657658
Oregon       -1.331981
Nevada       -1.000000
California    1.006778
Idaho        -1.000000
dtype: float64

### 範例：隨機取樣和排列

In [66]:
# 建立英文文字版本的一疊撲克牌
suits = ['H', 'S', 'C', 'D']   # 紅心、黑桃、梅花、方塊
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ['A'] + list(range(2, 11)) + ['J', 'Q', 'K']
cards = []
for suit in suits:
    cards.extend(str(num) + suit for num in base_names)
    
deck = pd.Series(card_val, index=cards)

In [67]:
deck

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
QH     10
KH     10
AS      1
2S      2
3S      3
4S      4
5S      5
6S      6
7S      7
8S      8
9S      9
10S    10
JS     10
QS     10
KS     10
AC      1
2C      2
3C      3
4C      4
5C      5
6C      6
7C      7
8C      8
9C      9
10C    10
JC     10
QC     10
KC     10
AD      1
2D      2
3D      3
4D      4
5D      5
6D      6
7D      7
8D      8
9D      9
10D    10
JD     10
QD     10
KD     10
dtype: int64

In [73]:
# 抽樣，隨機五張
def draw(deck, n=5):
    return deck.sample(n)

draw(deck)

QD    10
7S     7
6D     6
8H     8
2H     2
dtype: int64

In [74]:
# 每種花色隨機抽兩張牌
get_suit = lambda card: card[-1]  # 最後一個字是花色

deck.groupby(get_suit).apply(draw, n=2)

C  JC    10
   8C     8
D  4D     4
   KD    10
H  JH    10
   6H     6
S  4S     4
   9S     9
dtype: int64

In [76]:
# 同上效果
deck.groupby(get_suit, group_keys=False).apply(draw, n=2)

JC    10
QC    10
9D     9
6D     6
8H     8
QH    10
7S     7
AS     1
dtype: int64

### 範例：加權平均和關聯性

In [77]:
# 作用在 DataFrame或 兩個 Series之間
df = pd.DataFrame({'category': ['a', 'a', 'a', 'a', 'b', 'b', 'b', 'b'],
                   'data': np.random.randn(8),
                   'weights': np.random.randn(8)})
df

,category,data,weights
0,a,1.064323,-1.039802
1,a,2.137380,-0.613045
2,a,1.373259,1.150919
3,a,1.913301,0.205467
4,b,-1.741960,-0.457564
5,b,-0.326950,0.126418
6,b,0.745028,1.165950
7,b,1.420252,0.008004


In [81]:
# 對 category 做分組加權平均
grouped = df.groupby('category')
get_wavg = lambda g: np.average(g['data'], weights=g['weights'])   # weights:代表每個數字的重要性
grouped.apply(get_wavg)

category
a    1.495525
b    1.940844
dtype: float64

In [4]:
# Yahoo!Finance
# 包含股票、標準普爾500指數的每日收盤價
# ############################################
import requests

request_url = "https://raw.githubusercontent.com/wesm/pydata-book/2nd-edition/examples/stock_px_2.csv"
resp = requests.get(request_url)
# print(resp.status_code)
# print(resp.text)

resp_text = resp.text
file_name = request_url.split("/")[-1]

f = open(file_name, 'w')
f.write(resp_text)
f.close

<function TextIOWrapper.close()>

In [5]:
close_px = pd.read_csv('examples/stock_px_2.csv', parse_dates=True, index_col=0)

In [6]:
close_px.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AAPL    2214 non-null   float64
 1   MSFT    2214 non-null   float64
 2   XOM     2214 non-null   float64
 3   SPX     2214 non-null   float64
dtypes: float64(4)
memory usage: 86.5 KB


In [7]:
close_px[-4:]

,AAPL,MSFT,XOM,SPX
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66
2011-10-14,422.00,27.27,78.11,1224.58


In [8]:
# 各欄與'SPX'欄，兩兩抓對做關聯式分析
spx_corr = lambda x: x.corrwith(x['SPX'])

In [9]:
# 計算 close_px 的百分比變化
rets = close_px.pct_change().dropna()

In [11]:
# 以年來分組百分比的變化
get_year = lambda x: x.year

by_year = rets.groupby(get_year)
by_year.apply(spx_corr)

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


In [12]:
# 計算各個欄之間的關聯性, 如計算 AAPL和 MSFT每年之間的關聯性
by_year.apply(lambda g: g['AAPL'].corr(g['MSFT']))

2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

### 範例：分組線性回歸

In [14]:
!pip install statsmodels

Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the 'c:\program files (x86)\python38-64\python.exe -m pip install --upgrade pip' command.


In [18]:
import statsmodels.api as sm

def regress(data, yvar, xvars):
    Y = data[yvar]
    X = data[xvars]
    X['intercept'] = 1.
    result = sm.OLS(Y, X).fit()   # Ordinary Least Squares 最小平方法回歸
    return result.params

In [19]:
# AAPL 和 SPX 的年度線性回歸
by_year.apply(regress, 'AAPL', ['SPX'])

,SPX,intercept
2003,1.195406,0.000710
2004,1.363463,0.004201
2005,1.766415,0.003246
2006,1.645496,0.000080
2007,1.198761,0.003438
2008,0.968016,-0.001110
2009,0.879103,0.002954
2010,1.052608,0.001261
2011,0.806605,0.001514


## 10.4 樞紐關係表和交叉表

In [22]:
# DataFrame: pivot_table
# pandas: pandas.pivot_table

tips = pd.read_csv('examples/tips.csv')
tips['tip_pct'] = tips['tip'] / tips['total_bill']
tips.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


In [23]:
tips.pivot_table(index=['day', 'smoker'])

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

In [24]:
# 聚合 tip_pct 和 size 兩欄，並額外加上依時間分組
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'], columns='smoker')

size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

In [25]:
# 合計 margins
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'], columns='smoker', margins=True)

size                       tip_pct                    
smoker             No       Yes       All        No       Yes       All
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  0.139622  0.165347  0.158916
       Sat   2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
       Sun   2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
       Thur  2.000000       NaN  2.000000  0.159744       NaN  0.159744
Lunch  Fri   3.000000  1.833333  2.000000  0.187735  0.188937  0.188765
       Thur  2.500000  2.352941  2.459016  0.160311  0.163863  0.161301
All          2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

In [26]:
# 使用不同的聚合函式，指定給 aggfunc
tips.pivot_table('tip_pct', index=['time', 'smoker'], columns='day', aggfunc=len, margins=True)

day             Fri   Sat   Sun  Thur  All
time   smoker                             
Dinner No       3.0  45.0  57.0   1.0  106
       Yes      9.0  42.0  19.0   NaN   70
Lunch  No       1.0   NaN   NaN  44.0   45
       Yes      6.0   NaN   NaN  17.0   23
All            19.0  87.0  76.0  62.0  244

In [27]:
# 處理空值(或遺失值)
tips.pivot_table('tip_pct', index=['time', 'size', 'smoker'], columns='day', aggfunc='mean', fill_value=0)

day                      Fri       Sat       Sun      Thur
time   size smoker                                        
Dinner 1    No      0.000000  0.137931  0.000000  0.000000
            Yes     0.000000  0.325733  0.000000  0.000000
       2    No      0.139622  0.162705  0.168859  0.159744
            Yes     0.171297  0.148668  0.207893  0.000000
       3    No      0.000000  0.154661  0.152663  0.000000
            Yes     0.000000  0.144995  0.152660  0.000000
       4    No      0.000000  0.150096  0.148143  0.000000
            Yes     0.117750  0.124515  0.193370  0.000000
       5    No      0.000000  0.000000  0.206928  0.000000
            Yes     0.000000  0.106572  0.065660  0.000000
       6    No      0.000000  0.000000  0.103799  0.000000
Lunch  1    No      0.000000  0.000000  0.000000  0.181728
            Yes     0.223776  0.000000  0.000000  0.000000
       2    No      0.000000  0.000000  0.000000  0.166005
            Yes     0.181969  0.000000  0.000000  0.158843
       3    No      0.187735  0.000000  0.000000  0.084246
            Yes     0.000000  0.000000  0.000000  0.204952
       4    No      0.000000  0.000000  0.000000  0.138919
            Yes     0.000000  0.000000  0.000000  0.155410
       5    No      0.000000  0.000000  0.000000  0.121389
       6    No      0.000000  0.000000  0.000000  0.173706

### 交叉表：Crosstab

In [28]:
from io import StringIO
data = """\
Sample  Nationality  Handedness
1   USA  Right-handed
2   Japan    Left-handed
3   USA  Right-handed
4   Japan    Right-handed
5   Japan    Left-handed
6   Japan    Right-handed
7   USA  Right-handed
8   USA  Left-handed
9   Japan    Right-handed
10  USA  Right-handed"""
data = pd.read_table(StringIO(data), sep='\s+')

In [29]:
data

,Sample,Nationality,Handedness
0,1,USA,Right-handed
1,2,Japan,Left-handed
2,3,USA,Right-handed
3,4,Japan,Right-handed
4,5,Japan,Left-handed
5,6,Japan,Right-handed
6,7,USA,Right-handed
7,8,USA,Left-handed
8,9,Japan,Right-handed
9,10,USA,Right-handed


In [30]:
# 計算分組頻率
pd.crosstab(data.Nationality, data.Handedness, margins=True)

Handedness,Left-handed,Right-handed,All
Nationality,,,
Japan,2,3,5
USA,1,4,5
All,3,7,10


In [31]:
pd.crosstab([tips.time, tips.day], tips.smoker, margins=True)

smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244